In [2]:
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np
#import math
from math import sqrt

In [3]:
def remove_border(image, crop_pixels):
    # Load the image
    #image_path = 
    #image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Apply a binary threshold to separate the fingerprint from the background
    _, thresh = cv2.threshold(image, 10, 255, cv2.THRESH_BINARY)
    
    # Find contours of the thresholded image
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # If no contours are found, return the original image
    if not contours:
        return image
    
    # Find the bounding rectangle of the largest contour
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    
    # Adjust the coordinates to crop the specified number of pixels from the border
    x = max(x + crop_pixels, 0)  # Ensure x does not go negative
    y = max(y + crop_pixels, 0)  # Ensure y does not go negative
    w = max(w - 2 * crop_pixels, 0)  # Reduce width
    h = max(h - 2 * crop_pixels, 0)  # Reduce height
    
    # Crop the image to the adjusted bounding rectangle
    cropped_image = image[y:y+h, x:x+w]

    '''cv2.namedWindow('test', cv2.WINDOW_NORMAL)
    cv2.imshow('test', cropped_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()'''
    
    return cropped_image





In [4]:
def crop_based_on_intensity(image, threshold=240):
    # Convert to grayscale if necessary
    #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape) > 2 else image

    # Threshold the background
    _, mask = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY_INV)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        # Get the bounding box of the largest contour
        x, y, w, h = cv2.boundingRect(max(contours, key=cv2.contourArea))
        cropped_image = image[y:y+h, x:x+w]
        return cropped_image
    
    return image

In [5]:
def create_segmented_and_variance_images(im, w, threshold=.2):
    """
    Returns mask identifying the ROI. Calculates the standard deviation in each image block and threshold the ROI
    It also normalises the intesity values of
    the image so that the ridge regions have zero mean, unit standard
    deviation.
    :param im: Image
    :param w: size of the block
    :param threshold: std threshold
    :return: segmented_image
    """
    (y, x) = im.shape
    threshold = np.std(im)*threshold

    image_variance = np.zeros(im.shape)
    segmented_image = im.copy()
    mask = np.ones_like(im)

    for i in range(0, x, w):
        for j in range(0, y, w):
            box = [i, j, min(i + w, x), min(j + w, y)]
            block_stddev = np.std(im[box[1]:box[3], box[0]:box[2]])
            image_variance[box[1]:box[3], box[0]:box[2]] = block_stddev

    # apply threshold
    mask[image_variance < threshold] = 0

    # smooth mask with a open/close morphological filter
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(w*2, w*2))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # normalize segmented image
    segmented_image *= mask
    #im = normalise(im)
    mean_val = np.mean(im[mask==0])
    std_val = np.std(im[mask==0])
    norm_img = (im - mean_val)/(std_val)

    return segmented_image, norm_img, mask

In [6]:

from skimage.morphology import skeletonize as skelt
from skimage.morphology import thin
def skeletonizee(image_input):
    image = np.zeros_like(image_input)
    image[image_input == 0] = 1.0
    output = np.zeros_like(image_input)

    skeleton = skelt(image)

    output[skeleton] = 255
    cv2.bitwise_not(output, output)
    cv2.namedWindow('test', cv2.WINDOW_NORMAL)
    cv2.imshow('test', output)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return output

In [7]:
def clahe_contrast(a):
    clahe = cv2.createCLAHE(clipLimit=5.0)
    #b = a
    a = clahe.apply(a)#4
    '''cv2.namedWindow('test', cv2.WINDOW_NORMAL)
    cv2.imshow('test', a)
    cv2.waitKey(0)
    cv2.destroyAllWindows()'''
    return a

In [8]:
def normalize_pixel(x, v0, v, m, m0):
    """
    From Handbook of Fingerprint Recognition pg 133
    Normalize job used by Hong, Wan and Jain(1998)
    similar to https://pdfs.semanticscholar.org/6e86/1d0b58bdf7e2e2bb0ecbf274cee6974fe13f.pdf equation 21
    :param x: pixel value
    :param v0: desired variance
    :param v: global image variance
    :param m: global image mean
    :param m0: desired mean
    :return: normilized pixel
    """
    dev_coeff = sqrt((v0 * ((x - m)**2)) / v)
    return m0 + dev_coeff if x > m else m0 - dev_coeff

def normalize(im, m0, v0):
    m = np.mean(im)
    v = np.std(im) ** 2
    (y, x) = im.shape
    normilize_image = im.copy()
    for i in range(x):
        for j in range(y):
            normilize_image[j, i] = normalize_pixel(im[j, i], v0, v, m, m0)

    return normilize_image

In [9]:
def preprocessing_normal(path):
    #filee=file[:-4]
    a = cv2.imread(path, cv2.IMREAD_GRAYSCALE)#convert the image to grayscale #1
    #a = remove_border(a, 2)#2
        
    '''a = crop_based_on_intensity(a)#2
    print('Cropped')
    cv2.namedWindow('test', cv2.WINDOW_NORMAL)
    cv2.imshow('test', a)
    cv2.waitKey(0)
    cv2.destroyAllWindows()'''

    a = normalize(a.copy(), float(100), float(100))#2
    cv2.namedWindow('test', cv2.WINDOW_NORMAL)
    cv2.imshow('test', a)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    
    #b = clahe_contrast(a)
    #segment(3)
    #(a, normim, mask) = create_segmented_and_variance_images(a, 8, 0.2)
    #print(a.shape)

    #a = cv2.resize(a, (512, 512), interpolation=cv2.INTER_CUBIC) #inter_cubic phù hợp tăng kích thước ảnh 3
    #print(a.shape)
    #a = gabor_filter(a)
    a = clahe_contrast(a)#3
    cv2.namedWindow('test', cv2.WINDOW_NORMAL)
    cv2.imshow('test', a)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


    #4 binarization
    thresh, a= cv2.threshold(a, 255, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    a = crop_using_morphology(a)
    cv2.namedWindow('test', cv2.WINDOW_NORMAL)
    cv2.imshow('test', a)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    a = cv2.resize(a, (512, 512), interpolation=cv2.INTER_CUBIC) #5 inter_cubic phù hợp tăng kích thước ảnh
    cv2.namedWindow('test', cv2.WINDOW_NORMAL)
    cv2.imshow('test', a)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    print(a.shape)
    
    '''(a, normim, mask) = create_segmented_and_variance_images(a, 8, 0.2)
    cv2.namedWindow('test', cv2.WINDOW_NORMAL)
    cv2.imshow('test', a)
    cv2.waitKey(0)
    cv2.destroyAllWindows()'''

    #skeletonize(5)(chắc  bỏ)
    #a = skeletonizee(a)
    

    
       
    return a


In [1]:
def crop_using_morphology(image):
    # Check if the image is grayscale (1 channel) or color (3 channels)
    if len(image.shape) == 2:  # Grayscale image
        gray = image  # No conversion needed
    elif len(image.shape) == 3 and image.shape[2] == 3:  # Color image
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        raise ValueError("Invalid image format: The image must be either grayscale or color (BGR).")
    
    # Apply thresholding
    _, binary = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY_INV)
    
    # Morphological closing to connect ridge lines
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    closed = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    
    # Find contours and crop based on the largest contour
    contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        x, y, w, h = cv2.boundingRect(max(contours, key=cv2.contourArea))
        cropped_image = image[y:y+h, x:x+w]
        return cropped_image
    return image


In [35]:
def crop_using_itself(img1,img2):
    result = np.zeros_like(img1)
    print(img1.shape)
    print(img2.shape)
    # Lặp qua từng pixel
    for i in range(img1.shape[0]):
        for j in range(img1.shape[1]):
            if img1[i, j] < img2[i, j]:
                result[i, j] = img2[i, j]
            else:
                result[i, j] = 255

    return result

In [5]:
def crop_openandclose(img):
    kernel = np.ones((5, 5), np.uint8) 
    opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel) 
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel) 
    # Tìm các contour trong ảnh sau khi xử lý 
    contours, _ = cv2.findContours(closing, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Chọn contour lớn nhất đại diện cho vùng vân tay 
    largest_contour = max(contours, key=cv2.contourArea) 
    # Lấy bounding box của contour lớn nhất 
    x, y, w, h = cv2.boundingRect(largest_contour) 
    # Cắt ảnh dựa trên bounding box 
    cropped_image = img[y:y+h, x:x+w]
    return cropped_image

In [23]:
import cv2
def on_mouse(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print('Toa do: (%d, %d), Gia tri pixel: %d' % (x, y, a[y, x]))

# Đọc ảnh xám
a = cv2.imread("C:\\Users\\Admin\\FingerprintClassification\\testjuan\\r1.jpg", cv2.IMREAD_GRAYSCALE)
a = normalize(a, float(100), float(100))
#thresh, b= cv2.threshold(a, 255, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
#print(b)
#a = clahe_contrast(a)
#a = normalize(a, float(150), float(50))
thresh, a= cv2.threshold(a, 255, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
#a = crop_using_morphology(a)
#a = cv2.GaussianBlur(a, (3,3), 0)
# Tạo cửa sổ và liên kết hàm callback
a = crop_openandclose(a)
cv2.namedWindow('Anh xam')
cv2.setMouseCallback('Anh xam', on_mouse)

# Hiển thị ảnh
cv2.imshow('Anh xam', a)
cv2.waitKey(0)
cv2.destroyAllWindows()

IndexError: index 104 is out of bounds for axis 1 with size 92

In [14]:
#paths for images
# 'C:\\Users\\84965\\Documents\\Fingerprint_Recognition_GROUP8\\model\\anhhvantay\\R_f0002_05.png'
# C:\\Users\\84965\\Documents\\Fingerprint_Recognition_GROUP8\\test\\batch3_R_f0002_05.png

In [39]:
image_path = 'C:\\Users\\84965\\Documents\\Fingerprint_Recognition_GROUP8\\model\\anhhvantay\\R_f0002_05.png'
img = preprocessing_normal(image_path)
cv2.namedWindow('test', cv2.WINDOW_NORMAL)
cv2.imshow('test', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


(512, 512)


In [ ]:
image_folder = "C:\\Users\\Admin\\FingerprintClassification\\testjuan"
for filename in os.listdir(image_folder):
    image_path = os.path.join(image_folder, filename)
    img = preprocessing_normal(image_path)
    cv2.namedWindow('test', cv2.WINDOW_NORMAL)
    cv2.imshow('test', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)
(512, 512)


: 

In [16]:
image_link = 'C:\\Users\\Admin\\Fingerprint_Recognition_GROUP8\\khunglongxanh.jpg'
image = cv2.imread(image_link, cv2.IMREAD_GRAYSCALE)
    # Create a kernel for erosion and dilation
kernel = np.ones((3, 3), np.uint8)
skel = np.ones(image.shape, np.uint8)
    
    # Store the previous image to check if any changes happen
prev_image = np.copy(image)
    
while True:
        # Erode the image
    eroded = cv2.erode(image, kernel)
    
        # If erosion doesn't change the image, break the loop
    if np.array_equal(eroded, prev_image):
        break
        
        # Subtract eroded from the original image
    temp = cv2.dilate(eroded, kernel)
    temp = cv2.subtract(image, temp)
    
        # Update skeleton
    skel = cv2.bitwise_or(skel, temp)
    
        # Update image for the next iteration
    prev_image = np.copy(eroded)
    image[:] = eroded
cv2.namedWindow('test', cv2.WINDOW_NORMAL)
cv2.imshow('test', skel)
cv2.waitKey(0)
cv2.destroyAllWindows()
